# **AI Chatbots for Imaging Recommendations Aligned with ACR Guidelines**



In [5]:
!pip install llama-index==0.5.0 -q
!pip install gradio -q
!pip install PyPDF2 -q

In [3]:
import os
import sys
import glob
import gradio as gr
import openai
from llama_index import (
    GPTSimpleVectorIndex,
    LLMPredictor,
    SimpleDirectoryReader,
    ServiceContext,
)
from langchain.chat_models import ChatOpenAI


# Set the OpenAI API key. If unsure about obtaining the API key, refer to https://platform.openai.com/account/api-keys for more information.
# For estimating costs associated with index creation and chatbot usage, please visit: https://openai.com/pricing
os.environ["OPENAI_API_KEY"] = 'your_api_key_here'


# Set the folder path for ACR data and the rebuild index flag. This script doesn't include
# importing ACR guidelines or a predefined index due to license restrictions on ACR guidelines.
# For more information, refer to the guidelines on the ACR website.

FOLDER_PATH = "ACR"
REBUILD_INDEX = False

def setup_index(folder_path, rebuild_index):
    index_file_name = 'index_simple35.json'
    
    # Check if the folder path exists and contains PDF files
    if not os.path.exists(folder_path) or not glob.glob(f"{folder_path}/*.pdf"):
        print("The folder path either does not exist or does not contain PDF files.")
        return

    # Rebuild the index if rebuild_index is True or the index file does not exist
    if rebuild_index or not os.path.isfile(index_file_name):
        # Load documents from the folder path
        documents = SimpleDirectoryReader(folder_path).load_data()
        
        # Initialize the LLMPredictor with the ChatOpenAI model
        llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=512, request_timeout=120))
        
        # Create a service context for the LLMPredictor
        service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)
        
        # Create an index from the documents and save it to disk
        index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
        index.save_to_disk(index_file_name)

def get_combined_chatbot_response(input_text):
    # Format the input text
    input_text = f'Case: {input_text} Question: Is imaging for this Usually Appropriate and if yes, state precisely only what imaging modality is the most Appropriate and if contrast agent is needed, do not mention "May Be Appropriate" or "Usually Not Appropriate".)'
    
    # Initialize the LLMPredictor with the ChatOpenAI model used in accGPT
    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=512, request_timeout=120))
    
    # Create a service context for the LLMPredictor
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)

    # Load the index from disk
    index = GPTSimpleVectorIndex.load_from_disk('index_simple35.json')
    
    # Define the messages to be sent to the models without index
    messages = [
        {"role": "system", "content": "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible."},
        {"role": "user", "content": input_text},
    ]

    # accGPT approach: Query the index and get the response from Top 3 Text nodes using GPT-3.5-Turbo 
    response = index.query(input_text, response_mode="compact", service_context=service_context, similarity_top_k=3)
    output_accGPT = response.response.replace('\n', '\\n')

    # Get the response from GPT-3.5-Turbo
    response35 = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, timeout=120)
    output35 = response35.choices[0].message['content'].replace('\n', '\\n')
    
    # Get the response from GPT-4
    response4 = openai.ChatCompletion.create(model="gpt-4", messages=messages, timeout=120)
    output4 = response4.choices[0].message['content'].replace('\n', '\\n')
    
    # Combine the responses from all models
    answer = f"accGPT: {output_accGPT}\n\nGPT 3.5-Turbo: {output35}\n\nGPT 4: {output4}"

    return answer


def launch_interface(chatbot_function):
    iface = gr.Interface(
        fn=get_combined_chatbot_response,
        inputs=[gr.Textbox(lines=7, label="Enter your case")],
        outputs=gr.Textbox(lines=7, label="Imaging Recommendations"),
        title="AI Chatbots for Imaging Recommendations Aligned with ACR Guidelines"
    )

    # Launch the interface
    iface.launch(share=True, debug=False)

def main():
    setup_index(FOLDER_PATH, REBUILD_INDEX)
    launch_interface(get_combined_chatbot_response)

if __name__ == "__main__":
    main()


The folder path either does not exist or does not contain PDF files.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://79524902c723a5af4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
